In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer
#from nltk.tokenize import word_tokenize
import string

In [2]:
data = pd.read_csv('updated_reviews.csv')

In [3]:
data

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,predicted_sentiment
0,US,1797882.0,R3I2DHQBR577SS,B001ANOOOE,2102612.0,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5.0,0.0,0.0,N,Y,Five Stars,love excel sun block,2015-08-31,5
1,US,18381298.0,R1QNE9NQFJC2Y4,B0016J22EQ,106393691.0,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5.0,0.0,0.0,N,Y,Thank you Alba Bontanica!,great thing cream doesnt smell weird like chem...,2015-08-31,5
2,US,19242472.0,R3LIDG2Q4LJBAO,B00HU6UQAG,375449471.0,"Elysee Infusion Skin Therapy Elixir, 2oz.",Beauty,5.0,0.0,0.0,N,Y,Five Stars,great product im year old claim,2015-08-31,5
3,US,19551372.0,R3KSZHPAEVPEAL,B002HWS7RM,255651889.0,"Diane D722 Color, Perm And Conditioner Process...",Beauty,5.0,0.0,0.0,N,Y,GOOD DEAL!,use shower cap condit cap like theyr bulk save...,2015-08-31,4
4,US,14802407.0,RAI2OIG50KZ43,B00SM99KWU,116158747.0,Biore UV Aqua Rich Watery Essence SPF50+/PA+++...,Beauty,5.0,0.0,0.0,N,Y,this soaks in quick and provides a nice base f...,goto daili sunblock leav white cast clean plea...,2015-08-31,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,US,6985401.0,RLWQNDS7I46ST,B00TDJ5VTO,959170259.0,"Waterproof Bluetooth Speaker, Keedox Wireless ...",Mobile_Electronics,5.0,0.0,0.0,N,Y,cool,work greatbr get wet reason tobr gal water proof,2015-06-14,4
29996,US,127390.0,RIDE7LEW92C1U,B00S015EMK,231869191.0,"Game Day w/ GPLC & Pure PF3, Blue Bomb-Sicle 250g",Health & Personal Care,1.0,0.0,0.0,N,Y,One Star,old batch gameday color tast differ ive bought...,2015-08-31,3
29997,US,13150882.0,R2M3HM87H61ETP,B007A1XG5I,730919193.0,New Chapter Every Woman's One Daily 40 Plus Bo...,Health & Personal Care,5.0,3.0,4.0,N,Y,Amazon Prime is the best and most cost effecti...,take vitamin year make differ mental physicall...,2015-08-31,5
29998,US,4722303.0,RW8FGV1OTGXMK,B005NJLN86,70378667.0,Massachusetts Engineers Arch and Logo Short Sl...,Sports,5.0,0.0,0.0,N,Y,Five Stars,good cotton flawlessli sewb togeth good job,2015-08-31,4


# Amazingmeter

In [4]:
# sample dataframe
df = pd.DataFrame(data)

# create "Amazingmeter" column
df["Amazingmeter"] = ""
for i, row in df.iterrows():
    pid = row["product_id"]
    rating = row["star_rating"]
    votes = row["helpful_votes"]
    sentiment = row["predicted_sentiment"]
    
    if votes != 0:
        df.at[i, "Amazingmeter"] = rating
    else:
        df.at[i, "Amazingmeter"] = sentiment
        
# group by product_id and take average of Amazingmeter
df["Amazingmeter"] = df.groupby("product_id")["Amazingmeter"].transform(lambda x: round(x.mean()))

In [5]:
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,predicted_sentiment,Amazingmeter
0,US,1797882.0,R3I2DHQBR577SS,B001ANOOOE,2102612.0,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5.0,0.0,0.0,N,Y,Five Stars,love excel sun block,2015-08-31,5,5
1,US,18381298.0,R1QNE9NQFJC2Y4,B0016J22EQ,106393691.0,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5.0,0.0,0.0,N,Y,Thank you Alba Bontanica!,great thing cream doesnt smell weird like chem...,2015-08-31,5,5
2,US,19242472.0,R3LIDG2Q4LJBAO,B00HU6UQAG,375449471.0,"Elysee Infusion Skin Therapy Elixir, 2oz.",Beauty,5.0,0.0,0.0,N,Y,Five Stars,great product im year old claim,2015-08-31,5,5
3,US,19551372.0,R3KSZHPAEVPEAL,B002HWS7RM,255651889.0,"Diane D722 Color, Perm And Conditioner Process...",Beauty,5.0,0.0,0.0,N,Y,GOOD DEAL!,use shower cap condit cap like theyr bulk save...,2015-08-31,4,4
4,US,14802407.0,RAI2OIG50KZ43,B00SM99KWU,116158747.0,Biore UV Aqua Rich Watery Essence SPF50+/PA+++...,Beauty,5.0,0.0,0.0,N,Y,this soaks in quick and provides a nice base f...,goto daili sunblock leav white cast clean plea...,2015-08-31,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,US,6985401.0,RLWQNDS7I46ST,B00TDJ5VTO,959170259.0,"Waterproof Bluetooth Speaker, Keedox Wireless ...",Mobile_Electronics,5.0,0.0,0.0,N,Y,cool,work greatbr get wet reason tobr gal water proof,2015-06-14,4,5
29996,US,127390.0,RIDE7LEW92C1U,B00S015EMK,231869191.0,"Game Day w/ GPLC & Pure PF3, Blue Bomb-Sicle 250g",Health & Personal Care,1.0,0.0,0.0,N,Y,One Star,old batch gameday color tast differ ive bought...,2015-08-31,3,3
29997,US,13150882.0,R2M3HM87H61ETP,B007A1XG5I,730919193.0,New Chapter Every Woman's One Daily 40 Plus Bo...,Health & Personal Care,5.0,3.0,4.0,N,Y,Amazon Prime is the best and most cost effecti...,take vitamin year make differ mental physicall...,2015-08-31,5,5
29998,US,4722303.0,RW8FGV1OTGXMK,B005NJLN86,70378667.0,Massachusetts Engineers Arch and Logo Short Sl...,Sports,5.0,0.0,0.0,N,Y,Five Stars,good cotton flawlessli sewb togeth good job,2015-08-31,4,4


In [6]:
categories=np.unique(df["product_category"])
categories

array(['Beauty', 'Health & Personal Care', 'Major Appliances',
       'Mobile_Electronics', 'Pet Products', 'Sports'], dtype=object)

# Top Performing Category

In [43]:
grouped_data_category = df.groupby('product_category').agg({'Amazingmeter': 'mean'})
grouped_data_category = grouped_data_category.sort_values(by='Amazingmeter', ascending=False).head()
grouped_data_category

,Amazingmeter
product_category,
Beauty,4.0500
Sports,4.0226
Health & Personal Care,3.9860
Pet Products,3.9260
Mobile_Electronics,3.6620


# Top 5 Products for each category

## Beauty

In [8]:
#Group data by category
beauty_df = df[df['product_category'] == "Beauty"]
beauty_data = beauty_df[beauty_df['Amazingmeter'] == 5]
beauty_data['total_votes'] = beauty_data['total_votes'].astype(int)
beauty_data['helpful_votes'] = beauty_data['helpful_votes'].astype(int)
beauty_data["Votes Percentage"]=beauty_data["helpful_votes"]/beauty_data["total_votes"]
beauty_data=beauty_data.fillna(0)
new_beauty_data=beauty_data.loc[:,['product_id','product_category','helpful_votes','Votes Percentage','star_rating','predicted_sentiment','Amazingmeter']]
new_beauty_data

<ipython-input-8-98e5b1ec3a39>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beauty_data['total_votes'] = beauty_data['total_votes'].astype(int)
<ipython-input-8-98e5b1ec3a39>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beauty_data['helpful_votes'] = beauty_data['helpful_votes'].astype(int)
<ipython-input-8-98e5b1ec3a39>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,product_id,product_category,helpful_votes,Votes Percentage,star_rating,predicted_sentiment,Amazingmeter
0,B001ANOOOE,Beauty,0,0.000000,5.0,5,5
1,B0016J22EQ,Beauty,0,0.000000,5.0,5,5
2,B00HU6UQAG,Beauty,0,0.000000,5.0,5,5
4,B00SM99KWU,Beauty,0,0.000000,5.0,5,5
6,B001SYWTFG,Beauty,0,0.000000,5.0,5,5
...,...,...,...,...,...,...,...
4987,B00BTO1PBE,Beauty,0,0.000000,5.0,5,5
4990,B004JJQG4C,Beauty,0,0.000000,3.0,5,5
4992,B00ZGJSDYU,Beauty,1,1.000000,5.0,3,5
4995,B00V96B65W,Beauty,5,0.833333,5.0,4,5


In [9]:
from sklearn.preprocessing import MinMaxScaler
#Group by product_id and find mean for the rest of the columns
grouped_data_beauty = beauty_data.groupby('product_id').agg({'Votes Percentage': 'mean', 
                                                      'helpful_votes': 'mean','product_id': 'size'})
grouped_data_beauty.columns = [*grouped_data_beauty.columns[:-1], 'Number of Buyers']
scaler = MinMaxScaler()
grouped_data_beauty['helpful_votes'] = scaler.fit_transform(grouped_data_beauty[['helpful_votes']])
grouped_data_beauty['Number of Buyers'] = scaler.fit_transform(grouped_data_beauty[['Number of Buyers']])
#grouped_data_beauty


In [10]:
grouped_data_beauty['weighted_sum'] = grouped_data_beauty['Votes Percentage']*0.25 + grouped_data_beauty['Number of Buyers']*0.5 + grouped_data_beauty['helpful_votes']*0.25
# sort the DataFrame by the weighted sum in descending order and get the top 5 items
top_items_beauty = grouped_data_beauty.sort_values(by='weighted_sum', ascending=False).head()
top_items_beauty

,Votes Percentage,helpful_votes,Number of Buyers,weighted_sum
product_id,,,,
B00IDWP4IA,0.416667,0.006757,1.0,0.605856
B0018A32XS,0.166667,0.002252,1.0,0.542230
B008U8N26I,0.400000,0.005405,0.8,0.501351
B003UKM9CO,0.000000,0.000000,1.0,0.500000
B0095IMNI0,0.000000,0.000000,1.0,0.500000


In [11]:
#Check
product_id = 'B00BTO1PBE'
product_df = df[df['product_id'] == product_id]
product_df.reset_index()

,index,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,predicted_sentiment,Amazingmeter
0,4987,US,46965806.0,RFIKBI9ABPG3M,B00BTO1PBE,192575510.0,"Dr Sponge Facial Cleansing Sponge, Charcoal",Beauty,5.0,0.0,0.0,N,Y,Five Stars,bought one daughter sinc acn love,2015-08-31,5,5


## Health & Personal Care

In [12]:
#Group data by category
health_df = df[df['product_category'] == "Health & Personal Care"]
health_data = health_df[health_df['Amazingmeter'] == 5]
health_data['total_votes'] = health_data['total_votes'].astype(int)
health_data['helpful_votes'] = health_data['helpful_votes'].astype(int)
health_data["Votes Percentage"]=health_data["helpful_votes"]/health_data["total_votes"]
health_data=health_data.fillna(0)
new_health_data=health_data.loc[:,['product_id','product_category','helpful_votes','Votes Percentage','star_rating','predicted_sentiment','Amazingmeter']]
#new_health_data

<ipython-input-12-51737882b515>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  health_data['total_votes'] = health_data['total_votes'].astype(int)
<ipython-input-12-51737882b515>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  health_data['helpful_votes'] = health_data['helpful_votes'].astype(int)
<ipython-input-12-51737882b515>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [13]:
from sklearn.preprocessing import MinMaxScaler
#Group by product_id and find mean for the rest of the columns
grouped_data_health = health_data.groupby('product_id').agg({'Votes Percentage': 'mean', 
                                                      'helpful_votes': 'mean','product_id': 'size'})
grouped_data_health.columns = [*grouped_data_health.columns[:-1], 'Number of Buyers']
scaler = MinMaxScaler()
grouped_data_health['helpful_votes'] = scaler.fit_transform(grouped_data_health[['helpful_votes']])
grouped_data_health['Number of Buyers'] = scaler.fit_transform(grouped_data_health[['Number of Buyers']])
#grouped_data_health

In [14]:
grouped_data_health['weighted_sum'] = grouped_data_health['Votes Percentage']*0.25 + grouped_data_health['Number of Buyers']*0.5 + grouped_data_health['helpful_votes']*0.25
# sort the DataFrame by the weighted sum in descending order and get the top 5 items
top_items_health = grouped_data_health.sort_values(by='weighted_sum', ascending=False).head()
top_items_health

,Votes Percentage,helpful_votes,Number of Buyers,weighted_sum
product_id,,,,
B00EIWU2FE,0.937500,0.005236,0.777778,0.624573
B00FMB4864,0.000000,0.000000,1.000000,0.500000
B0065ZTKWS,0.964646,1.000000,0.000000,0.491162
B009RV7ORK,0.965714,0.884817,0.000000,0.462633
B00A39MPNI,0.833333,0.007330,0.444444,0.432388


In [15]:
#Check
product_id = 'B00A39MPNI'
product_df = df[df['product_id'] == product_id]
product_df.reset_index()

,index,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,predicted_sentiment,Amazingmeter
0,15127,US,1844817.0,R2XNR0QSCEDQK0,B00A39MPNI,252371412.0,Turmeric Curcumin C3® Complex; Turmeric Suppor...,Health & Personal Care,4.0,1.0,1.0,N,Y,Four Stars,good way get tumer liquid without strong tast,2015-08-31,4,5
1,15146,US,914721.0,R1UB5KQ4Q32YE8,B00A39MPNI,252371412.0,Turmeric Curcumin C3® Complex; Turmeric Suppor...,Health & Personal Care,4.0,2.0,2.0,N,Y,BEST PRODUCT,sixti chunki bare move say bare move didnt eve...,2015-08-31,4,5
2,17474,US,37379422.0,R3QN6PN9R9THGM,B00A39MPNI,252371412.0,Turmeric Curcumin C3® Complex; Turmeric Suppor...,Health & Personal Care,5.0,1.0,2.0,N,Y,An Excellent Product at a Great Price,ive heard great thing turmer brother far infla...,2015-08-31,5,5
3,17777,US,17607365.0,R2OQT6RLRUWM9S,B00A39MPNI,252371412.0,Turmeric Curcumin C3® Complex; Turmeric Suppor...,Health & Personal Care,5.0,1.0,1.0,N,Y,High quality product,great product qualiti product impress taken mu...,2015-08-31,5,5
4,19589,US,49713792.0,RON3BAHZ887V7,B00A39MPNI,252371412.0,Turmeric Curcumin C3® Complex; Turmeric Suppor...,Health & Personal Care,5.0,2.0,3.0,N,Y,5 Star Product for Tendonitis,friend recommend turmer tendon research onlin ...,2015-08-31,5,5


## Major Appliances

In [16]:
#Group data by category
appliance_df = df[df['product_category'] == "Major Appliances"]
appliance_data = appliance_df[appliance_df['Amazingmeter'] == 5]
appliance_data['total_votes'] = appliance_data['total_votes'].astype(int)
appliance_data['helpful_votes'] = appliance_data['helpful_votes'].astype(int)
appliance_data["Votes Percentage"]=appliance_data["helpful_votes"]/appliance_data["total_votes"]
appliance_data=appliance_data.fillna(0)
new_appliance_data=appliance_data.loc[:,['product_id','product_category','helpful_votes','Votes Percentage','star_rating','predicted_sentiment','Amazingmeter']]
#new_appliance_data

<ipython-input-16-95d55b237c4d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appliance_data['total_votes'] = appliance_data['total_votes'].astype(int)
<ipython-input-16-95d55b237c4d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appliance_data['helpful_votes'] = appliance_data['helpful_votes'].astype(int)
<ipython-input-16-95d55b237c4d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [17]:
from sklearn.preprocessing import MinMaxScaler
#Group by product_id and find mean for the rest of the columns
grouped_data_appliance = appliance_data.groupby('product_id').agg({'Votes Percentage': 'mean', 
                                                      'helpful_votes': 'mean','product_id': 'size'})
grouped_data_appliance.columns = [*grouped_data_appliance.columns[:-1], 'Number of Buyers']
scaler = MinMaxScaler()
grouped_data_appliance['helpful_votes'] = scaler.fit_transform(grouped_data_appliance[['helpful_votes']])
grouped_data_appliance['Number of Buyers'] = scaler.fit_transform(grouped_data_appliance[['Number of Buyers']])
#grouped_data_appliance

In [18]:
grouped_data_appliance['weighted_sum'] = grouped_data_appliance['Votes Percentage']*0.25 + grouped_data_appliance['Number of Buyers']*0.5 + grouped_data_appliance['helpful_votes']*0.25
# sort the DataFrame by the weighted sum in descending order and get the top 5 items
top_items_appliance = grouped_data_appliance.sort_values(by='weighted_sum', ascending=False).head()
top_items_appliance

,Votes Percentage,helpful_votes,Number of Buyers,weighted_sum
product_id,,,,
B00L1JT4YM,0.883333,0.006835,1.000000,0.722542
B00MVVITWC,0.778571,0.017536,0.789474,0.593764
B0002YTM0I,0.294118,0.003386,0.842105,0.495428
B005ECTRE4,0.972028,1.000000,0.000000,0.493007
B00MEOD2C8,0.983740,0.870504,0.000000,0.463561


In [19]:
#Check
product_id = 'B00L1JT4YM'
product_df = df[df['product_id'] == product_id]
product_df.reset_index()

,index,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,predicted_sentiment,Amazingmeter
0,5043,US,12488325.0,R1EWS25SGGC2L1,B00L1JT4YM,839318065.0,Samsung DA29-00020B Compatible Water Filter - ...,Major Appliances,5.0,1.0,1.0,N,Y,Five Stars,thank,2015-08-31,5,5
1,5414,US,30782422.0,R2VOM5F1EBN6SF,B00L1JT4YM,839318065.0,Samsung DA29-00020B Compatible Water Filter - ...,Major Appliances,5.0,1.0,1.0,N,Y,Five Stars,great filter cheaper samsung buy,2015-08-28,5,5
2,5460,US,42542681.0,R7SZR2XAK95EM,B00L1JT4YM,839318065.0,Samsung DA29-00020B Compatible Water Filter - ...,Major Appliances,4.0,1.0,1.0,N,Y,Four Stars,seam work fine one month,2015-08-27,2,5
3,5463,US,22653475.0,R29RH1Y223SVYK,B00L1JT4YM,839318065.0,Samsung DA29-00020B Compatible Water Filter - ...,Major Appliances,5.0,1.0,1.0,N,Y,Five Stars,perfect,2015-08-27,5,5
4,5625,US,11740960.0,R2EAAN2W3NU2MW,B00L1JT4YM,839318065.0,Samsung DA29-00020B Compatible Water Filter - ...,Major Appliances,5.0,1.0,1.0,N,Y,Five Stars,work great order anoth pleas,2015-08-26,5,5
5,6933,US,10904432.0,R2Q2LYDZI1Z3WI,B00L1JT4YM,839318065.0,Samsung DA29-00020B Compatible Water Filter - ...,Major Appliances,5.0,1.0,1.0,N,Y,Its a filter,filter water,2015-08-16,3,5
6,7064,US,41059148.0,R1N1734RANJ58K,B00L1JT4YM,839318065.0,Samsung DA29-00020B Compatible Water Filter - ...,Major Appliances,1.0,0.0,1.0,N,Y,bad taste and leaking fridge!,filter list compat refriger fit ok water tast ...,2015-08-15,2,5
7,7346,US,18518845.0,R11PI8N76CU2XH,B00L1JT4YM,839318065.0,Samsung DA29-00020B Compatible Water Filter - ...,Major Appliances,5.0,1.0,1.0,N,Y,Great product,arriv time easi instal water doesnt tast great...,2015-08-12,5,5
8,7548,US,34204836.0,R3FLM4PYDQ0F8N,B00L1JT4YM,839318065.0,Samsung DA29-00020B Compatible Water Filter - ...,Major Appliances,5.0,1.0,1.0,N,Y,Five Stars,expect work perfectli,2015-08-11,3,5
9,7603,US,16717459.0,R2AX4KOD08XPSI,B00L1JT4YM,839318065.0,Samsung DA29-00020B Compatible Water Filter - ...,Major Appliances,5.0,2.0,3.0,N,Y,The filter works just as well as the Samsung f...,filter work well samsung filter remov although...,2015-08-11,2,5


## Mobile_Electronics

In [41]:
#Group data by category
electronics_df = df[df['product_category'] == "Mobile_Electronics"]
electronics_data = electronics_df[electronics_df['Amazingmeter'] == 5]
electronics_data['total_votes'] = electronics_data['total_votes'].astype(int)
electronics_data['helpful_votes'] = electronics_data['helpful_votes'].astype(int)
electronics_data["Votes Percentage"]=electronics_data["helpful_votes"]/electronics_data["total_votes"]
electronics_data=electronics_data.fillna(0)
new_electronics_data=electronics_data.loc[:,['product_id','product_category','helpful_votes','Votes Percentage','star_rating','predicted_sentiment','Amazingmeter']]
new_electronics_data

<ipython-input-41-6196d036097e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  electronics_data['total_votes'] = electronics_data['total_votes'].astype(int)
<ipython-input-41-6196d036097e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  electronics_data['helpful_votes'] = electronics_data['helpful_votes'].astype(int)
<ipython-input-41-6196d036097e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

,product_id,product_category,helpful_votes,Votes Percentage,star_rating,predicted_sentiment,Amazingmeter
9999,B00OQMFG1Q,Mobile_Electronics,0,0.0,5.0,5,5
10005,B00ET5AWBY,Mobile_Electronics,0,0.0,5.0,5,5
10006,B00Z9K379C,Mobile_Electronics,0,0.0,4.0,5,5
10010,B00M49C6MK,Mobile_Electronics,0,0.0,5.0,5,5
10015,B009NGR0AK,Mobile_Electronics,1,1.0,5.0,5,5
...,...,...,...,...,...,...,...
14976,B00GYUZ4SI,Mobile_Electronics,0,0.0,4.0,5,5
14981,B004GIN0UO,Mobile_Electronics,0,0.0,5.0,5,5
14982,B00H4IUEJI,Mobile_Electronics,2,1.0,5.0,5,5
14988,B00RLECSXK,Mobile_Electronics,1,1.0,5.0,4,5


In [21]:
from sklearn.preprocessing import MinMaxScaler
#Group by product_id and find mean for the rest of the columns
grouped_data_electronics = electronics_data.groupby('product_id').agg({'Votes Percentage': 'mean', 
                                                      'helpful_votes': 'mean','product_id': 'size'})
grouped_data_electronics.columns = [*grouped_data_electronics.columns[:-1], 'Number of Buyers']
scaler = MinMaxScaler()
grouped_data_electronics['helpful_votes'] = scaler.fit_transform(grouped_data_electronics[['helpful_votes']])
grouped_data_electronics['Number of Buyers'] = scaler.fit_transform(grouped_data_electronics[['Number of Buyers']])
#grouped_data_electronics

In [22]:
grouped_data_electronics['weighted_sum'] = grouped_data_electronics['Votes Percentage']*0.25 + grouped_data_electronics['Number of Buyers']*0.5 + grouped_data_electronics['helpful_votes']*0.25
# sort the DataFrame by the weighted sum in descending order and get the top 5 items
top_items_electronics = grouped_data_electronics.sort_values(by='weighted_sum', ascending=False).head()
top_items_electronics

,Votes Percentage,helpful_votes,Number of Buyers,weighted_sum
product_id,,,,
B00M49C6MK,0.055556,0.001522,1.000000,0.514269
B00YEMAT38,0.935897,1.000000,0.000000,0.483974
B00U05MVPM,0.909770,0.019839,0.451613,0.458209
B00XJB8522,1.000000,0.136986,0.016129,0.292311
B010W4OF4K,0.827586,0.328767,0.000000,0.289088


In [23]:
#Check
product_id = 'B00M49C6MK'
product_df = df[df['product_id'] == product_id]
product_df.reset_index()

,index,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,predicted_sentiment,Amazingmeter
0,10010,US,49359994.0,RLIL6S9OGM3YA,B00M49C6MK,91176851.0,Sentey Ls 4162,Mobile_Electronics,5.0,0.0,0.0,N,Y,Five Stars,love,2015-08-31,5,5
1,10098,US,15909878.0,RVL6X6XTAV2OC,B00M49C6MK,91176851.0,Sentey Ls 4162,Mobile_Electronics,5.0,0.0,0.0,N,Y,Five Stars,thank,2015-08-30,5,5
2,10372,US,924194.0,R141C1YSNM3V1E,B00M49C6MK,91176851.0,Sentey Ls 4162,Mobile_Electronics,5.0,0.0,0.0,N,Y,Five Stars,great sound impress love speaker,2015-08-25,5,5
3,10441,US,45947289.0,R23XVIG5RJ3GWC,B00M49C6MK,91176851.0,Sentey Ls 4162,Mobile_Electronics,5.0,0.0,0.0,N,Y,Five Stars,great product loud,2015-08-24,5,5
4,10511,US,12050162.0,R3JNDCBPB7VY83,B00M49C6MK,91176851.0,Sentey Ls 4162,Mobile_Electronics,5.0,0.0,0.0,N,Y,Great buy!,great sound quit loud your look featur easi se...,2015-08-23,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,14565,US,10343818.0,R24GJ2MA4S723,B00M49C6MK,91176851.0,Sentey Ls 4162,Mobile_Electronics,4.0,1.0,2.0,N,N,Great sound,great small speaker got discount code review l...,2015-06-20,5,5
59,14655,US,13377470.0,R3NXQ8TDROZQDO,B00M49C6MK,91176851.0,Sentey Ls 4162,Mobile_Electronics,5.0,0.0,0.0,N,N,Such a nice portable speaker,nice portabl speaker conveni carri car pool go...,2015-06-19,5,5
60,14677,US,4934271.0,RZIZ0NC5GHWY6,B00M49C6MK,91176851.0,Sentey Ls 4162,Mobile_Electronics,5.0,0.0,0.0,N,N,This thing thumps!,well made solid great sound speaker youd hard ...,2015-06-18,5,5
61,14795,US,10180378.0,R2J020O042GNN9,B00M49C6MK,91176851.0,Sentey Ls 4162,Mobile_Electronics,5.0,0.0,0.0,N,N,"This speaker is amazing! I own all three ""size...",speaker amaz three size bluetooth speaker sent...,2015-06-17,5,5


## Pet Products

In [24]:
#Group data by category
pet_df = df[df['product_category'] == "Pet Products"]
pet_data = pet_df[pet_df['Amazingmeter'] == 5]
pet_data['total_votes'] = pet_data['total_votes'].astype(int)
pet_data['helpful_votes'] = pet_data['helpful_votes'].astype(int)
pet_data["Votes Percentage"]=pet_data["helpful_votes"]/pet_data["total_votes"]
pet_data=pet_data.fillna(0)
new_pet_data=pet_data.loc[:,['product_id','product_category','helpful_votes','Votes Percentage','star_rating','predicted_sentiment','Amazingmeter']]
#new_pet_data

<ipython-input-24-f3f04a1c0e53>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pet_data['total_votes'] = pet_data['total_votes'].astype(int)
<ipython-input-24-f3f04a1c0e53>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pet_data['helpful_votes'] = pet_data['helpful_votes'].astype(int)
<ipython-input-24-f3f04a1c0e53>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [25]:
from sklearn.preprocessing import MinMaxScaler
#Group by product_id and find mean for the rest of the columns
grouped_data_pet = pet_data.groupby('product_id').agg({'Votes Percentage': 'mean', 
                                                      'helpful_votes': 'mean','product_id': 'size'})
grouped_data_pet.columns = [*grouped_data_pet.columns[:-1], 'Number of Buyers']
scaler = MinMaxScaler()
grouped_data_pet['helpful_votes'] = scaler.fit_transform(grouped_data_pet[['helpful_votes']])
grouped_data_pet['Number of Buyers'] = scaler.fit_transform(grouped_data_pet[['Number of Buyers']])
#grouped_data_pet

In [26]:
grouped_data_pet['weighted_sum'] = grouped_data_pet['Votes Percentage']*0.25 + grouped_data_pet['Number of Buyers']*0.5 + grouped_data_pet['helpful_votes']*0.25
# sort the DataFrame by the weighted sum in descending order and get the top 5 items
top_items_pet = grouped_data_pet.sort_values(by='weighted_sum', ascending=False).head()
top_items_pet

,Votes Percentage,helpful_votes,Number of Buyers,weighted_sum
product_id,,,,
B00TF4DHGG,0.750000,0.011076,1.000000,0.690269
B00XIKSN3K,0.908046,1.000000,0.000000,0.477011
B011AY4JWO,0.990991,0.312236,0.285714,0.468664
B0027IVL6K,1.000000,0.012658,0.428571,0.467450
B0048ZAKLW,1.000000,0.835443,0.000000,0.458861


In [27]:
#Check
product_id = 'B00TF4DHGG'
product_df = df[df['product_id'] == product_id]
product_df.reset_index()

,index,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,predicted_sentiment,Amazingmeter
0,25649,US,33375209.0,R1BDLOVO0VT0R7,B00TF4DHGG,100398455.0,Plush Paws Ultra-Luxury Pet Seat Cover - 2 Har...,Pet Products,5.0,1.0,1.0,N,Y,Five Stars,need,2015-08-31,4,5
1,25812,US,21494121.0,R30X0RBFH9BFE6,B00TF4DHGG,100398455.0,Plush Paws Ultra-Luxury Pet Seat Cover - 2 Har...,Pet Products,5.0,1.0,1.0,N,Y,Five Stars,smooth transact prompt shipmentlov seat cover ...,2015-08-31,4,5
2,26370,US,47657443.0,R2AK3A5IG186IY,B00TF4DHGG,100398455.0,Plush Paws Ultra-Luxury Pet Seat Cover - 2 Har...,Pet Products,5.0,1.0,1.0,N,Y,Plush Paws Pet Seat Cover- A great deal!,great qualiti appreci extra good instruct reco...,2015-08-31,5,5
3,26550,US,51823706.0,R1TBVT972GZTFA,B00TF4DHGG,100398455.0,Plush Paws Ultra-Luxury Pet Seat Cover - 2 Har...,Pet Products,5.0,1.0,2.0,N,Y,REALLY love this seat cover,realli love seat cover found cover onlin searc...,2015-08-31,4,5
4,28423,US,17983596.0,R33LJULGH776SR,B00TF4DHGG,100398455.0,Plush Paws Ultra-Luxury Pet Seat Cover - 2 Har...,Pet Products,5.0,1.0,1.0,N,Y,Well worth the price!!,actual blown away amaz qualiti seat cover anch...,2015-08-31,5,5
5,28723,US,41380244.0,R1UCDF6K6ZH1H4,B00TF4DHGG,100398455.0,Plush Paws Ultra-Luxury Pet Seat Cover - 2 Har...,Pet Products,5.0,0.0,0.0,N,Y,Great product,great product deliv time would order,2015-08-31,5,5
6,29686,US,25408445.0,R3W431OSBA3KVQ,B00TF4DHGG,100398455.0,Plush Paws Ultra-Luxury Pet Seat Cover - 2 Har...,Pet Products,5.0,1.0,2.0,N,Y,A great product and great customer service!,bought seat cover usual use old beach towel la...,2015-08-30,5,5
7,29959,US,26781828.0,R2J5N23Y3TYABV,B00TF4DHGG,100398455.0,Plush Paws Ultra-Luxury Pet Seat Cover - 2 Har...,Pet Products,5.0,1.0,1.0,N,N,The long awaited has so far proven beyond meas...,long await final arrivedbr return long holiday...,2015-08-30,2,5


## Sports

In [28]:
#Group data by category
sports_df = df[df['product_category'] == "Sports"]
sports_data = sports_df[sports_df['Amazingmeter'] == 5]
sports_data['total_votes'] = sports_data['total_votes'].astype(int)
sports_data['helpful_votes'] = sports_data['helpful_votes'].astype(int)
sports_data["Votes Percentage"]=sports_data["helpful_votes"]/sports_data["total_votes"]
sports_data=sports_data.fillna(0)
new_sports_data=sports_data.loc[:,['product_id','product_category','helpful_votes','Votes Percentage','star_rating','predicted_sentiment','Amazingmeter']]
#new_sports_data

<ipython-input-28-3d4e5e2a2e9a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sports_data['total_votes'] = sports_data['total_votes'].astype(int)
<ipython-input-28-3d4e5e2a2e9a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sports_data['helpful_votes'] = sports_data['helpful_votes'].astype(int)
<ipython-input-28-3d4e5e2a2e9a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [29]:
from sklearn.preprocessing import MinMaxScaler
#Group by product_id and find mean for the rest of the columns
grouped_data_sports = sports_data.groupby('product_id').agg({'Votes Percentage': 'mean', 
                                                      'helpful_votes': 'mean','product_id': 'size'})
grouped_data_sports.columns = [*grouped_data_sports.columns[:-1], 'Number of Buyers']
scaler = MinMaxScaler()
grouped_data_sports['helpful_votes'] = scaler.fit_transform(grouped_data_sports[['helpful_votes']])
grouped_data_sports['Number of Buyers'] = scaler.fit_transform(grouped_data_sports[['Number of Buyers']])
#grouped_data_sports

In [30]:
grouped_data_sports['weighted_sum'] = grouped_data_sports['Votes Percentage']*0.25 + grouped_data_sports['Number of Buyers']*0.5 + grouped_data_sports['helpful_votes']*0.25
# sort the DataFrame by the weighted sum in descending order and get the top 5 items
top_items_sports = grouped_data_sports.sort_values(by='weighted_sum', ascending=False).head()
top_items_sports

,Votes Percentage,helpful_votes,Number of Buyers,weighted_sum
product_id,,,,
B003TJH3LI,0.000000,0.000000,1.0,0.500000
B00HMDLAPC,0.968750,1.000000,0.0,0.492188
B00NG1ECBS,0.714286,0.005760,0.6,0.480012
B00S0YA5GM,0.886598,0.693548,0.0,0.395037
B00ATIE5ZE,1.000000,0.008065,0.2,0.352016


In [31]:
#Check
product_id = 'B00NG1ECBS'
product_df = df[df['product_id'] == product_id]
product_df.reset_index()

,index,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,predicted_sentiment,Amazingmeter
0,20282,US,42341403.0,R3IC01WQX0K3QJ,B00NG1ECBS,565346728.0,PowerNet Baseball and Softball Practice Net 7 ...,Sports,5.0,0.0,0.0,N,Y,Great net for backyard!!,great product son use everi day,2015-08-31,5,5
1,21508,US,27540823.0,R3NXSAYE9AT1YR,B00NG1ECBS,565346728.0,PowerNet Baseball and Softball Practice Net 7 ...,Sports,5.0,1.0,1.0,N,Y,Very Easy to Setup Great Product,boy age play basebal year old set net practic ...,2015-08-31,5,5
2,22320,US,21208051.0,R1YOCSQL5ZP8AY,B00NG1ECBS,565346728.0,PowerNet Baseball and Softball Practice Net 7 ...,Sports,5.0,1.0,1.0,N,Y,Five Stars,excel product,2015-08-31,5,5
3,22571,US,50188686.0,R3QJ8I9AL1F3OU,B00NG1ECBS,565346728.0,PowerNet Baseball and Softball Practice Net 7 ...,Sports,5.0,1.0,1.0,N,Y,Best Value Bow Style Net For Baseball/Softball...,awesom product everi bit good brand name net l...,2015-08-31,5,5
4,23310,US,13937632.0,R267D6TKFPEVBF,B00NG1ECBS,565346728.0,PowerNet Baseball and Softball Practice Net 7 ...,Sports,5.0,1.0,1.0,N,Y,It was easy to put together and works great,powernet arriv time day suppos easi put togeth...,2015-08-31,5,5
5,23475,US,28098324.0,R3M4SCQQZQKF6O,B00NG1ECBS,565346728.0,PowerNet Baseball and Softball Practice Net 7 ...,Sports,5.0,0.0,0.0,N,Y,Net is great!,great product,2015-08-31,5,5
6,24904,US,44092731.0,RFFIP6EG3B2PC,B00NG1ECBS,565346728.0,PowerNet Baseball and Softball Practice Net 7 ...,Sports,5.0,1.0,1.0,N,Y,We love the Powernet!,exactli like bownet cost lot less easi assembl...,2015-08-31,5,5


# Top Buyers

In [32]:
buyers_cnt = df.groupby('customer_id').agg({'customer_id': 'size'})
buyers_cnt.columns = [*buyers_cnt.columns[:-1], 'Count']

buyers_cnt.head()

,Count
customer_id,
10182.0,1
12598.0,2
16511.0,1
20968.0,1
21087.0,3


In [33]:
buyers_cnt = df.groupby('customer_id').agg({'customer_id': 'size'})
buyers_cnt.columns = [*buyers_cnt.columns[:-1], 'Count']
top_buyers = buyers_cnt.sort_values(by='Count', ascending=False).head()
top_buyers.head()

,Count
customer_id,
21959897.0,42
23208852.0,29
48872127.0,26
12612039.0,26
11599687.0,20


In [34]:
#Check
customer_id = 53089047
customer_df = df[df['customer_id'] == customer_id]
customer_df.reset_index()

,index,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,predicted_sentiment,Amazingmeter
0,9249,US,53089047.0,RM7E0PXNE9JH,B00KJ07SEM,313983847.0,GE MWF SmartWater Compatible Water Filter Cart...,Major Appliances,5.0,0.0,0.0,N,Y,Five Stars,great price easi instal,2015-07-28,5,4
